In [ ]:
# Copyright 2023, Acadential, All rights reserved.

# 6-8. Pytorch로 구현하는 Gradient Descent

## Automatic Differentiation: pytorch의 backpropagation
### (각 layer의 gradient에 대한 계산)

- PyTorch에서는 built-in differentiation engine인 `torch.autograd`을 통해서 Backpropagation을 수행할 수 있습니다.
- 이를 통하여 model weight (parameter)들에 대한 `Loss Gradient`을 구할 수 있습니다.
- 예를 들어서 저희가 `PyTorch Module Class`의 `forward` 함수를 실행하면
     - 해당 함수를 구성하는 연산들이 차례대로 수행되면서
     - 해당 연산들에 대한 `Computational Graph`가 생성됩니다.
- 이렇게 정의된 `Computation Graph`에 대해서 backpropagation을 수행하여 gradient을 계산할 수 있습니다.

## Computation Graph의 예시

다음과 같은 computational graph가 있다고 가정해보겠습니다. 

- input = x 
- parameter = W, b 
- activation = tanh 
- loss function = MSE Loss 

```
L = L(y_gt, y_pred)
y_pred = activation(W @ x + b)
```

In [1]:
import torch

x = torch.rand(10)
y = torch.zeros(5)
W = torch.rand(10, 5, requires_grad=True)
b = torch.rand(5, requires_grad=True)
h = torch.matmul(x, W) + b
h = torch.tanh(h)
loss = torch.sum(torch.square(y - h))

## Computing Gradients

- ```Output.backward()```을 실행하면 ```Output tensor```을 각 ```parameter```에 대해서 미분한 경사들을 자동으로 계산해줍니다.
- ```d(Output)/d(tensor)```은 ```tensor.grad```에 누적됩니다.
- 만약에 기존에 ```.backward```로 계산된 Loss가 있다면 새로운 gradient는 기존의 gradient에 더해집니다.

In [2]:
loss.backward()
print(W.grad)
print(b.grad)

tensor([[0.0036, 0.0014, 0.0012, 0.0020, 0.0105],
        [0.0672, 0.0269, 0.0217, 0.0365, 0.1970],
        [0.0043, 0.0017, 0.0014, 0.0023, 0.0125],
        [0.0423, 0.0169, 0.0136, 0.0230, 0.1241],
        [0.0759, 0.0304, 0.0245, 0.0412, 0.2225],
        [0.0869, 0.0347, 0.0280, 0.0472, 0.2547],
        [0.0177, 0.0071, 0.0057, 0.0096, 0.0518],
        [0.0581, 0.0232, 0.0187, 0.0316, 0.1704],
        [0.0442, 0.0177, 0.0142, 0.0240, 0.1296],
        [0.0238, 0.0095, 0.0077, 0.0129, 0.0699]])
tensor([0.0976, 0.0390, 0.0314, 0.0530, 0.2860])


## 참고 사항:
- PyTorch에서 by default로 ```backward(gradient=None)```은 scalar value에 대해서만 수행할 수 있습니다!
- 물론 Vector, Matrix, 혹은 Tensor에 대해서도 backward을 수행해 줄 수는 있지만 이때는 별도로, 각 element들에 대한 Gradient을 명시해줘야 합니다.

### 에러 예시

In [3]:
x = torch.rand(10)  # input tensor
y = torch.zeros(5)  # expected output
W = torch.randn(10, 5, requires_grad=True)
b = torch.randn(5, requires_grad=True)
h = torch.matmul(x, W)+b
h = torch.tanh(h)
loss = torch.square(y - h)  # Raises an error! Because loss is not scalar
print("loss shape == ", loss.shape)
loss.backward()

loss shape ==  torch.Size([5])


RuntimeError: grad can be implicitly created only for scalar outputs

### (앞서 살펴본) 잘 작동되는 예시

torch.sum을 통해서 loss을 scalar 값으로 구했습니다.

In [4]:
x = torch.rand(10)  # input tensor
y = torch.zeros(5)  # expected output
W = torch.randn(10, 5, requires_grad=True)
b = torch.randn(5, requires_grad=True)
h = torch.matmul(x, W)+b
h = torch.tanh(h)
loss = torch.sum(torch.square(y - h))
print("loss shape == ", loss.shape) 
loss.backward()

loss shape ==  torch.Size([])


## Disabling Gradient Tracking

PyTorch에서는 by default로 backward propagation을 수행하는데 필요한 값(각 layer에서의 intermediate values)들을 저장해둡니다.

d(Leaf)/d(Root) gradient을 구하기 위해서 Leaf (Descendent node)로부터 Root node까지 backpropagation하려면:
1. 각 layer의 output값
2. 각 layer의 input 값

들을 저장해둬야합니다 (gradient을 구하기 위해 필요한 값).

하지만 만약에 training 단계가 아니라 inference (혹은 evaluation)단계의 경우 backpropagation이 필요하지 않습니다.

따라서 inference의 경우 각 layer의 output값과 input값을 굳이 저장할 필요가 없고 Gradient을 계산할 필요도 없습니다.

그러므로 inference에서는 torch.no_grad()을 사용해서 Gradient에 대한 tracking을 disable합니다.

In [6]:
z = torch.matmul(x, W)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, W)+b
print(z.requires_grad)

True
False


### To detach a tensor from computational graph (disabling gradient computation)

또 간혹 pre-trained된 모델을 가져와서 finetuning하는데 사용할때 layer 전부 다 학습하기보다 일부 (예를 들어 feature extractor)의 parameter들은 고정된 상태 (freeze)로 학습을 하고 싶은 경우가 있을 수 있습니다.

이 때는 freeze할 weight parameter들에 대해서는 .detach()해서 gradient에 대해서는 gradient tracking을 disable시킵니다.

In [8]:
z = torch.matmul(x, W)+b
z_det = z.detach()
print(z_det.requires_grad)

False


## Zeroing the gradient


.backward()을 여러번 실행하게 되면 해당 tensor에 대한 gradient은 "여러번 쌓이게" 됩니다! \
즉, 기존에 계산한 gradient에 더해집니다. \
따라서 의도된 것이 아니라면, gradient을 "zero" (비워줘야) 해당 oepration에 대한 gradient을 구할 수 있습니다.


In [9]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)  # operation == (x+1) ** 2 -> gradient == 2(x+1)

In [10]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)  # operation == (x+1) ** 2 -> gradient == 2(x+1)
print(inp)

out.backward(torch.ones_like(inp), retain_graph=True)
print(f"First call\n{inp.grad}")

out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")

inp.grad.zero_()
print(f"\nGradient after zeroing\n{inp.grad}")

out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)
First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Gradient after zeroing
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
